In [8]:
import ctypes
from pathlib import Path
import pandas as pd

In [9]:

class MapsForgeHeader(ctypes.BigEndianStructure):
    _pack_ = 1
    _fields_ = [
        ("magic_byte", ctypes.c_char * 20),
        ("header_size", ctypes.c_uint32),
        ("file_version", ctypes.c_uint32),
        ("file_size", ctypes.c_uint64),
        ("date_of_creation", ctypes.c_uint64),
        ("minLat", ctypes.c_uint32),
        ("minLon", ctypes.c_uint32),
        ("maxLat", ctypes.c_uint32),  # most nothern point
        ("maxLon", ctypes.c_uint32),
        ("tile_size", ctypes.c_uint16),
    ]


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseHeader(file: Path) -> MapsForgeHeader:
    header = MapsForgeHeader.from_buffer_copy(file.read_bytes())
    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)


In [10]:
files = [f for f in Path("./igps_maps").glob(("*.map")) if f.is_file()]
data = []


print("--- files ----")
for i in files:
    print(i.name)


--- files ----
Carinthia-AT02002303103DR27W01K00Q.map
Burgenland-AT01002303103FV26Y00R019.map
Vorarlberg-AT08002303103BS27H00H00P.map
UpperAustria-AT04002303103DT26C01G01A.map
Tyrol-AT07002303103C527B01U012.map
Steiermark-AT06002303103EC27801O016.map
LowerAustria-AT03002303103EW26301O01K.map
Vienna-AT09002303103G026S009007.map
Salzburg-AT05002303103DE271018012.map


In [11]:
for f in files:
    header = parseHeader(f)
    # fix modified name (my maps are named like Vorarlberg-AT08002303103BS27H00H00P.map)
    humanName, filename = f.name.split("-")
    pf = parseFilename(filename)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": humanName})

In [12]:
print("--- bounding boxes from mapsforge files ---")

for entry in data:
    print(
        entry["Name"].ljust(13),
        "minLat",
        entry["header"].minLat,
        "minLon",
        str(entry["header"].minLon).ljust(9),
        "maxLat",
        entry["header"].maxLat,
        "maxLon",
        entry["header"].maxLon,
    )


--- bounding boxes from mapsforge files ---
Carinthia     minLat 46350000 minLon 12635000  maxLat 47150000 maxLon 15085000
Burgenland    minLat 46810000 minLon 15975000  maxLat 48140000 maxLon 17180000
Vorarlberg    minLat 46837000 minLon 9527000   maxLat 47600000 maxLon 10241000
UpperAustria  minLat 47440000 minLon 12730000  maxLat 48795000 maxLon 15010000
Tyrol         minLat 46630000 minLon 10080000  maxLat 47765000 maxLon 12985000
Steiermark    minLat 46590000 minLon 13545000  maxLat 47850000 maxLon 16190000
LowerAustria  minLat 47400000 minLon 14435000  maxLat 49040000 maxLon 17090000
Vienna        minLat 48114000 minLon 16178000  maxLat 48327000 maxLon 16582000
Salzburg      minLat 46925000 minLon 12055000  maxLat 48060000 maxLon 14015000


In [ ]:
print("--- sorted by maxLat, using field 2 from filename ---")
data = sorted(data, key=lambda entry: entry["header"].maxLat, reverse=True)
prevmaxLatHeader = 0
prevmaxLatFilename = 0
for entry in data:
    maxLatHeader = entry["header"].maxLat
    maxLatFilename = entry["parsedFilename"].field2

    diffHeader =  maxLatHeader - prevmaxLatHeader
    diffFileName = maxLatFilename - prevmaxLatFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLatHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLatHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLatFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLatFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLatHeader != diffHeader else " --- ".ljust(9)
    )
    prevmaxLatHeader = maxLatHeader
    prevmaxLatFilename = maxLatFilename



--- sorted by maxLat, using field 1 from filename ---
LowerAustria  from header: 49040000  diff  ---      filename: 2811      diff  ---      diff factor  ---     
UpperAustria  from header: 48795000  diff -245000   filename: 2820      diff 9         diff factor -27222.222222222223
Vienna        from header: 48327000  diff -468000   filename: 2836      diff 16        diff factor -29250.0
Burgenland    from header: 48140000  diff -187000   filename: 2842      diff 6         diff factor -31166.666666666668
Salzburg      from header: 48060000  diff -80000    filename: 2845      diff 3         diff factor -26666.666666666668
Steiermark    from header: 47850000  diff -210000   filename: 2852      diff 7         diff factor -30000.0
Tyrol         from header: 47765000  diff -85000    filename: 2855      diff 3         diff factor -28333.333333333332
Vorarlberg    from header: 47600000  diff -165000   filename: 2861      diff 6         diff factor -27500.0
Carinthia     from header: 47150000  

In [ ]:
print("--- sorted by minLon---")
data = sorted(data, key=lambda entry: entry["header"].minLon, reverse=True)
prevmaxLonHeader = 0
prevmaxLonFilename = 0
for entry in data:
    maxLonHeader = entry["header"].minLon
    maxLonFilename = entry["parsedFilename"].field1

    diffHeader =  maxLonHeader - prevmaxLonHeader
    diffFileName = maxLonFilename - prevmaxLonFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLonHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLonHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLonFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLonFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLonHeader != diffHeader else " --- ".ljust(9)
    )
    prevmaxLonHeader = maxLonHeader
    prevmaxLonFilename = maxLonFilename


--- sorted by minLon---
Vienna        from header: 16178000  diff  ---      filename: 4464      diff  ---      diff factor  ---     
Burgenland    from header: 15975000  diff -203000   filename: 4459      diff -5        diff factor 40600.0
LowerAustria  from header: 14435000  diff -1540000  filename: 4424      diff -35       diff factor 44000.0
Steiermark    from header: 13545000  diff -890000   filename: 4404      diff -20       diff factor 44500.0
UpperAustria  from header: 12730000  diff -815000   filename: 4385      diff -19       diff factor 42894.73684210526
Carinthia     from header: 12635000  diff -95000    filename: 4383      diff -2        diff factor 47500.0
Salzburg      from header: 12055000  diff -580000   filename: 4370      diff -13       diff factor 44615.38461538462
Tyrol         from header: 10080000  diff -1975000  filename: 4325      diff -45       diff factor 43888.88888888889
Vorarlberg    from header: 9527000   diff -553000   filename: 4312      diff -13       d